In [95]:
import os
import json
from typing import Final
from yfpy.query import YahooFantasySportsQuery
from dotenv import load_dotenv
import pandas as pd
import uuid

auth_dir = "/Users/petepritchard/Documents/Projects/ff-league-app/bot/auth"

# Load environment variables from .env file
load_dotenv()

# Fetch Yahoo API credentials from environment variables
CONSUMER_KEY: Final[str] = os.getenv('CONSUMER_KEY')
CONSUMER_SECRET: Final[str] = os.getenv('CONSUMER_SECRET')
REFRESH_TOKEN: Final[str] = os.getenv('REFRESH_TOKEN')

# Fetch league information
query = YahooFantasySportsQuery(
    auth_dir, 
    league_id='49754', # Fill in league_id
    game_code='nfl',   # Leave this as is
    game_id='423',     # Fill in current game id
    consumer_key=CONSUMER_KEY,
    consumer_secret=CONSUMER_SECRET,
    )

In [27]:
transactions = query.get_league_transactions()
recent_transaction = transactions[1].to_json()
data = json.loads(recent_transaction)

# Retrieving the values
player_id = data['players']['player']['player_id']
destination_team_key = data['players']['player']['transaction_data']['destination_team_key']
transaction_type = data['players']['player']['transaction_data']['type']

# Printing the retrieved values
print("Player ID:", player_id)
print("Destination Team Key:", destination_team_key)
print("Transaction Type:", transaction_type)

## Pulling data from Yahoo for Matchups Table

In [71]:
# Initialize an empty list to store records
records = []

for i in range(1, 3):
    # Query data from yahoo
    matchups = query.get_league_matchups_by_week(i)
    # Loop through matchup data
    for i in range(len(matchups)):
        matchup = matchups[i].to_json() # matchup[i] is a yfpy object
        matchup_dict = json.loads(matchup)
        
        # Generate a unique matchup_id (UUID)
        matchup_id = str(uuid.uuid4())

        # Extract information for each team in the matchup
        for team in matchup_dict.get('teams', []):
            team_info = team.get('team', {})
            record = {
                'primary_key': str(uuid.uuid4()),  # Unique primary key for each row
                'matchup_id': matchup_id,
                'season_key': team_info.get('team_key')[:3], # First 3 characters in team key
                'week': matchup_dict.get('week'),
                'is_consolation': matchup_dict.get('is_consolation'),
                'is_playoffs': matchup_dict.get('is_playoffs'),
                'is_tied': matchup_dict.get('is_tied'),
                'winner_team_key': matchup_dict.get('winner_team_key'),
                'team_key': team_info.get('team_key'),
                'team_total_points': team_info.get('team_points', {}).get('total'),
                'team_projected_points': team_info.get('team_projected_points', {}).get('total')
            }
            
            # Add the record to the list
            records.append(record)

# Convert the list of records to a DataFrame
df = pd.DataFrame(records)

# Display the DataFrame
print(df.head())

                            primary_key                            matchup_id  \
0  423bb7d1-1c33-4b36-b457-156a7124b87e  6dbf59fa-4e35-4170-adab-1527a3f11f19   
1  78de06a1-d6be-466d-b0e0-ce97fae9677e  6dbf59fa-4e35-4170-adab-1527a3f11f19   
2  943c5aff-a0c8-4d92-aae1-ae69045ac7b6  aed76cc7-3954-4bd5-afcf-6ce4601b3827   
3  b0e3b5c7-8b64-480e-aaea-3d176c9f89bf  aed76cc7-3954-4bd5-afcf-6ce4601b3827   
4  f6f1900c-20c9-4615-b75f-5f5a421ef108  618208f4-129d-40ad-9ed0-c9f30c153b8a   

  season_key  week  is_consolation  is_playoffs  is_tied  winner_team_key  \
0        423     1               0            0        0  423.l.49754.t.7   
1        423     1               0            0        0  423.l.49754.t.7   
2        423     1               0            0        0  423.l.49754.t.2   
3        423     1               0            0        0  423.l.49754.t.2   
4        423     1               0            0        0  423.l.49754.t.3   

          team_key  team_total_points  team_projec

In [69]:
df.to_clipboard()

In [ ]:
matchups_table = """
CREATE TABLE Matchups (
    primary_key INT AUTO_INCREMENT PRIMARY KEY,
    matchup_id INT,
    season_key INT NOT NULL,
    week INT NOT NULL,
    is_consolation BOOLEAN,
    is_playoffs BOOLEAN,
    is_tied BOOLEAN,
    winner_team_key VARCHAR(50),
    team_key VARCHAR(50),
    team_total_points INT,
    team_projected_points INT,
    UNIQUE(season, week, winner_team_key) -- Ensuring unique combination of season, week, and winner team
);
"""

## Seasons Table

In [94]:
seasons_dict = {
    "season_key": ['257', '273', '314', '331', '348', '259', '317', '380', '380', '399', '406', '414', '423', '449'],
    "season": ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024'],
}

df_seasons = pd.DataFrame(seasons_dict)
df_seasons.head()

,season_key,season
0,257,2011
1,273,2012
2,314,2013
3,331,2014
4,348,2015


In [ ]:
seasons_table = """
CREATE TABLES Seasons (
    season_key INT PRIMARY KEY,
    season INT
    UNIQUE(season_key, season) -- Ensuring unique combination of season and season_key
);
"""

## Drafts Table

In [99]:
draft = query.get_league_draft_results()
print(type(draft[1]))

<class 'yfpy.models.DraftResult'>


In [100]:
print(draft[1])

DraftResult({
  "pick": 2,
  "round": 1,
  "team_key": "423.l.49754.t.6",
  "player_key": "423.p.33393"
})
